# Diagnostics vent géostrophique, divergence et tourbillon

Auteur : FERRY Frédéric (DESR/ENM/C3M) - novembre 2021

On utilise ici les données quotidiennes NCEP de géopotentiel, vent zonal, vent méridien et vitesse verticale pour l'année 2016 : 
https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/pressure/hgt.2016.nc
https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/pressure/uwnd.2016.nc
https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/pressure/vwnd.2016.nc
https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/pressure/omega.2016.nc

Concepts illustrés :

- Tracé de champs de géopotentiel et de vent horizontal
- Diagnostic du vent géostrophique à partir du champ de géopotentiel
- Calcul du vent agéostrophique et lien avec les vitesses verticales de grande échelle
- Calcul de la divergence du vent
- Calcul de la composante verticale du tourbillon
- Tracé de cartes (cartopy)

<div class="alert alert-danger">
<b>En rouge : </b> code à compléter
</div>

<div class="alert alert-warning">
<b>En jaune : </b> questions
</div>

In [1]:
import os

import numpy as np

import xarray as xr

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from mpl_toolkits.axes_grid1 import AxesGrid

import shapely.geometry as sgeom
from area import area

import warnings
warnings.filterwarnings('ignore')

In [2]:
diri="./data/"
dir_figs="./figs/"
if not os.path.exists(dir_figs):
    os.makedirs(dir_figs)

In [3]:
re=6.37e6
g=9.81
omega=7.292e-5

# Ouverture des données

In [4]:
year='2016'
month='01'
day='15'
date=year+'-'+month+'-'+day

In [5]:
wlev=600
ulev=int(input("Choix du niveau vertical pour le vent (300, 850): "))
zlev=int(input("Choix du niveau vertical pour le géopotentiel (300, 850) : "))

Choix du niveau vertical pour le vent (300, 850): 300
Choix du niveau vertical pour le géopotentiel (300, 850) : 300


In [6]:
#--  Read z, t, u, v, w data for specified vertical level
fz    = xr.open_dataset(diri+'hgt.'+year+'.nc').sel(level=zlev).sel(time=date).squeeze()
fu    = xr.open_dataset(diri+'uwnd.'+year+'.nc').sel(level=ulev).sel(time=date).squeeze()
fv    = xr.open_dataset(diri+'vwnd.'+year+'.nc').sel(level=ulev).sel(time=date).squeeze()
fw    = xr.open_dataset(diri+'omega.'+year+'.nc').sel(level=wlev).sel(time=date).squeeze()

print(fz)
print(fu)
print(fv)
print(fw)

#--  Extract variables
z0 = fz['hgt']
u0 = fu['uwnd']
v0 = fv['vwnd']
w0 = fw['omega']
print(z0.shape)

<xarray.Dataset>
Dimensions:    (lat: 73, lon: 144, nbnds: 2)
Coordinates:
    level      float32 300.0
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    time       datetime64[ns] 2016-01-15
Dimensions without coordinates: nbnds
Data variables:
    hgt        (lat, lon) float32 ...
    time_bnds  (nbnds) float64 ...
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (2014)
    history:        created 2013/12 by Hoop (netCDF2.3)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    References:     http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reana...
    dataset_title:  NCEP-NCAR Reanalysis 1
<xarray.Dataset>
Dimensions:    (lat: 73, lon: 144, nbnds: 2)
Coordinates:
    level      float32 300.0
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon      

In [7]:
def lonflip(da):
    lon_name = 'lon'
    da['_longitude_adjusted'] = xr.where(
        da[lon_name] > 180,
        da[lon_name] - 360,
        da[lon_name])
    da = (
        da
        .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(da._longitude_adjusted)})
        .drop(lon_name))
    da = da.rename({'_longitude_adjusted': lon_name})
    return da

In [8]:
z1 = lonflip(z0)
u1 = lonflip(u0)
v1 = lonflip(v0)
w1 = lonflip(w0)

lon0  = z1.lon.values
print(lon0)

[-177.5 -175.  -172.5 -170.  -167.5 -165.  -162.5 -160.  -157.5 -155.
 -152.5 -150.  -147.5 -145.  -142.5 -140.  -137.5 -135.  -132.5 -130.
 -127.5 -125.  -122.5 -120.  -117.5 -115.  -112.5 -110.  -107.5 -105.
 -102.5 -100.   -97.5  -95.   -92.5  -90.   -87.5  -85.   -82.5  -80.
  -77.5  -75.   -72.5  -70.   -67.5  -65.   -62.5  -60.   -57.5  -55.
  -52.5  -50.   -47.5  -45.   -42.5  -40.   -37.5  -35.   -32.5  -30.
  -27.5  -25.   -22.5  -20.   -17.5  -15.   -12.5  -10.    -7.5   -5.
   -2.5    0.     2.5    5.     7.5   10.    12.5   15.    17.5   20.
   22.5   25.    27.5   30.    32.5   35.    37.5   40.    42.5   45.
   47.5   50.    52.5   55.    57.5   60.    62.5   65.    67.5   70.
   72.5   75.    77.5   80.    82.5   85.    87.5   90.    92.5   95.
   97.5  100.   102.5  105.   107.5  110.   112.5  115.   117.5  120.
  122.5  125.   127.5  130.   132.5  135.   137.5  140.   142.5  145.
  147.5  150.   152.5  155.   157.5  160.   162.5  165.   167.5  170.
  172.5  175.   177.

In [9]:
latS = 30
latN = 70
lonW = -90
lonE = 20

z=z1.sel(lat=slice(latN,latS)).sel(lon=slice(lonW,lonE))
u=u1.sel(lat=slice(latN,latS)).sel(lon=slice(lonW,lonE))
v=v1.sel(lat=slice(latN,latS)).sel(lon=slice(lonW,lonE))
w=w1.sel(lat=slice(latN,latS)).sel(lon=slice(lonW,lonE))

print(z.shape)

lon  = z.lon.values
lat  = z.lat.values

print('Latitudes : ', lat)
print('Longitudes : ', lon)

(17, 45)
Latitudes :  [70.  67.5 65.  62.5 60.  57.5 55.  52.5 50.  47.5 45.  42.5 40.  37.5
 35.  32.5 30. ]
Longitudes :  [-90.  -87.5 -85.  -82.5 -80.  -77.5 -75.  -72.5 -70.  -67.5 -65.  -62.5
 -60.  -57.5 -55.  -52.5 -50.  -47.5 -45.  -42.5 -40.  -37.5 -35.  -32.5
 -30.  -27.5 -25.  -22.5 -20.  -17.5 -15.  -12.5 -10.   -7.5  -5.   -2.5
   0.    2.5   5.    7.5  10.   12.5  15.   17.5  20. ]


# Diagnostics

In [10]:
xlon,ylat=np.meshgrid(lon,lat)

print(xlon.shape) # lon x lat
print(ylat.shape) # lon x lat

#print(ylat[0,0])
#print(ylat[1,0])
#print(xlon[0,0])
#print(xlon[0,1])

(17, 45)
(17, 45)


In [11]:
dlony,dlonx=np.gradient(xlon)
dlaty,dlatx=np.gradient(ylat)

#print(dlonx)
#print(dlaty)

$$dx = a cos(\phi)d\lambda$$
$$dy = a d\phi$$

In [12]:
dx=re*np.cos(ylat*np.pi/180)*dlonx*np.pi/180
dy=re*dlaty*np.pi/180

<div class="alert alert-danger">
<b>Complétez le code avec le calcul du paramètre de Coriolis (f) : </b>
$$f = 2\Omega sin(\phi)$$
</div>

In [15]:
f = 2*omega*np.sin(ylat*np.pi/180)
print(f.shape)

(17, 45)


<div class="alert alert-danger">
<b>Complétez le code avec le calcul du vent géostrophique (ug, vg) : </b>
$$u_g = -\frac{g}{f} \frac{\partial z}{\partial y}$$
$$v_g = \frac{g}{f} \frac{\partial z}{\partial x}$$
</div>

<div class="alert alert-danger">
<b>Complétez le code avec le calcul du vent agéostrophique (ua, va) : </b>
$$u_a = u - u_g$$
$$v_a = v - v_g$$
</div>

In [16]:
# calculate derivatives
dzdx=np.gradient(z,axis=1)/dx
dzdy=np.gradient(z,axis=0)/dy

# calculate geostrophic wind
ug = (-1)*(g/f)*dzdy
vg = (g/f)*dzdx

# calculate ageostrophic wind
ua = u-ug
va = v-vg

<div class="alert alert-danger">
<b>Complétez le code avec le calcul des modules des vents (horizontal, géostrophique, agéostrophique). </b>
</div>

In [ ]:
hwind_mod = 
gwind_mod = 
awind_mod = 

<div class="alert alert-danger">
<b>Complétez le code avec le calcul de la divergence horizontale (div) et de la divergence géostrophique (divg) :</b>
$$div(\vec{V}_h) = \frac{\partial u}{\partial x}+\frac{\partial v}{\partial y}-\frac{v}{a}tan(\phi)$$
$$div(\vec{V}_g) = \frac{\partial u_g}{\partial x}+\frac{\partial v_g}{\partial y}-\frac{v_g}{a}tan(\phi)$$
</div>

<div class="alert alert-danger">
<b>Complétez le code avec le calcul du tourbillon relatif (vort) et du tourbillon géostrophique (vortg) :</b>
$$\xi = \frac{\partial v}{\partial x}-\frac{\partial u}{\partial y}+\frac{u}{a}tan(\phi)$$
$$\xi_g = \frac{\partial v_g}{\partial x}-\frac{\partial u_g}{\partial y}+\frac{u_g}{a}tan(\phi)$$
</div>

<div class="alert alert-danger">
<b>Complétez le code avec le calcul de la divergence agéostrophique (diva) et du tourbillon agéostrophique (vorta) : </b>
$$div(\vec{V}_a) = \frac{\partial u_a}{\partial x}+\frac{\partial v_a}{\partial y}-\frac{v_a}{a}tan(\phi)$$
$$\xi_a = \frac{\partial v_a}{\partial x}-\frac{\partial u_a}{\partial y}+\frac{u_a}{a}tan(\phi)$$
</div>

In [ ]:
# calculate derivatives
dudx=np.gradient(u,axis=1)/dx
dvdx=np.gradient(v,axis=1)/dx
dudy=np.gradient(u,axis=0)/dy
dvdy=np.gradient(v,axis=0)/dy

dugdx=np.gradient(ug,axis=1)/dx
dvgdx=np.gradient(vg,axis=1)/dx
dugdy=np.gradient(ug,axis=0)/dy
dvgdy=np.gradient(vg,axis=0)/dy

duadx=np.gradient(ua,axis=1)/dx
dvadx=np.gradient(va,axis=1)/dx
duady=np.gradient(ua,axis=0)/dy
dvady=np.gradient(va,axis=0)/dy

# Divergence and vorticity
div =
divg =
diva =

vort =
vortg =
vorta =

# Paramètres de tracés de cartes

In [ ]:
def plot_background(ax):
    ax.coastlines()
    ax.set_xticks(np.linspace(-180, 180, 19), crs=projection)
    ax.set_yticks(np.linspace(-90, 90, 19), crs=projection)
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    return ax

In [ ]:
levels_z = np.arange(round(int(z.min()),-1),round(int(z.max()),-1),50)
levels_w = np.arange(-0.5,0.6,0.1)
levels_wpos = np.arange(0,0.4,0.05)
levels_wneg = np.arange(-0.4,0,0.05)

if ulev==300:
    levels_wind = np.arange(30,72.5,2.5)
if ulev==850:
    levels_wind = np.arange(10,26,1)
levels_div = np.arange(-20,21,1)
levels_vor = np.arange(-10,11,1)
levels_f = np.arange(0,2.1,0.1)

wind_slice = slice(None, None, 2)

#projection = ccrs.Orthographic(central_longitude=0, central_latitude=45)
projection = ccrs.PlateCarree()
bounds = [(lonW, lonE, latS, latN)]

# Domaine et paramètre de Coriolis

In [ ]:
fig = plt.figure(figsize=(15., 8.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Coriolis parameter', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, f*10**4, levels_f, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('10$^{-4}$ s$^{-1}$')
c = ax.contour(lon, lat, f*10**4, levels_f, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1f', fontsize=10)

figname=dir_figs+'Coriolis_'+date
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

# Lien masse-vent

In [ ]:
fig = plt.figure(figsize=(15., 10.))
fig.suptitle('Geopotential height and winds at '+str(ulev)+' hPa : '+date, fontsize=16)

ax = fig.add_subplot(211, projection=projection)
ax.set_title('Geopotential height and horizontal wind at '+str(ulev)+' hPa', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())           
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                u[wind_slice, wind_slice], v[wind_slice, wind_slice],
                color='red', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

ax = fig.add_subplot(212, projection=projection)
ax.set_title('Geopotential height and geostrophic wind at '+str(ulev)+' hPa', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())              
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                ug[wind_slice, wind_slice], vg[wind_slice, wind_slice],
                color='red', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

figname=dir_figs+'Z_vh_vg_'+str(zlev)+'_'+date
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b>A grande échelle, au niveau isobare choisi, le vent géostrophique est-il une bonne approximation du vent horizontal ?</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

# Comparaison du vent horizontal et du vent géostrophique

In [ ]:
fig = plt.figure(figsize=(15., 10.))
fig.suptitle('Geopotential height and winds at '+str(ulev)+' hPa : '+date, fontsize=16)

ax = fig.add_subplot(211, projection=projection)
ax.set_title('Geopotential height and horizontal wind at '+str(ulev)+' hPa', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())               
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                u[wind_slice, wind_slice], v[wind_slice, wind_slice],
                color='red', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

ax = fig.add_subplot(212, projection=projection)
ax.set_title('Geopotential height and streamlines at '+str(ulev)+' hPa', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
c2= ax.streamplot(lon, lat, np.array(u), np.array(v), transform=ccrs.PlateCarree(),
                  linewidth=1, density=2, color=np.array(hwind_mod))

figname=dir_figs+'Z_wind_'+str(zlev)+'_'+date
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b>Le vent horizontal est-il parfaitement tangent aux isohypses ? Qu'est-ce que cela traduit-il ?</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

In [ ]:
fig = plt.figure(figsize=(15., 10.))
fig.suptitle('Geopotential height and winds at '+str(ulev)+' hPa : '+date, fontsize=16)

ax = fig.add_subplot(211, projection=projection)
ax.set_title('Geopotential height, horizontal wind and horizontal wind module at '+str(ulev)+' hPa',
             loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())    
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
cf = ax.contourf(lon, lat, hwind_mod, levels_wind, transform=ccrs.PlateCarree(), cmap='Spectral_r', extend='max')
cb = fig.colorbar(cf, orientation='vertical')
cb.set_label('m/s')          
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                u[wind_slice, wind_slice], v[wind_slice, wind_slice],
                color='red', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

ax = fig.add_subplot(212, projection=projection)
ax.set_title('Geopotential height, geostrophic wind and geostrophic wind module at '+str(ulev)+' hPa',
             loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())           
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
cf = ax.contourf(lon, lat, gwind_mod, levels_wind, transform=ccrs.PlateCarree(), cmap='Spectral_r', extend='max')
cb = fig.colorbar(cf, orientation='vertical')
cb.set_label('m/s')
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                ug[wind_slice, wind_slice], vg[wind_slice, wind_slice],
                color='red', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

figname=dir_figs+'Z_vh_vg_mod_'+str(zlev)+'_'+date
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Dans les courbures cycloniques, l'intensité du vent géostrophique est-elle supérieure ou inférieure à celle du vent horizontal ?</p>
<p><b>2) </b>Même question pour les courbures anticycloniques.</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

# Ecart entre vent horizontal et vent géostrophique (vent agéostrophique)

In [ ]:
fig = plt.figure(figsize=(10., 17.))
fig.suptitle('Geopotential height and winds at '+str(ulev)+' hPa : '+date, fontsize=16)

ax = fig.add_subplot(411, projection=projection)
ax.set_title('Geopotential height and horizontal wind at '+str(ulev)+' hPa', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())       
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                u[wind_slice, wind_slice], v[wind_slice, wind_slice],
                color='red', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

ax = fig.add_subplot(412, projection=projection)
ax.set_title('Geopotential height and geostrophic wind at '+str(ulev)+' hPa', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())               
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                ug[wind_slice, wind_slice], vg[wind_slice, wind_slice],
                color='red', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

ax = fig.add_subplot(413, projection=projection)
ax.set_title('Geopotential height and ageostrophic wind at '+str(ulev)+' hPa', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())    
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                ua[wind_slice, wind_slice], va[wind_slice, wind_slice],
                color='red', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

ax = fig.add_subplot(414, projection=projection)
ax.set_title('Geopotential height at '+str(ulev)+ 'hPa and vertical motion at '+str(wlev)+ 'hPa', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())            
c1 = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c1, fmt='%4.1i', fontsize=10)
c2 = ax.contour(lon, lat, w, levels_wpos[levels_wpos != 0], colors='brown', linewidths=1,
                transform=ccrs.PlateCarree()) 
c3 = ax.contour(lon, lat, w, levels_wneg[levels_wneg != 0], colors='green', linewidths=1,
                linestyle="--", transform=ccrs.PlateCarree()) 
ax.clabel(c2, fmt='%4.2f', fontsize=10)
ax.clabel(c3, fmt='%4.2f', fontsize=10)


figname=dir_figs+'Z_vh_vg_va_'+str(zlev)+'_w_'+str(wlev)+'_'+date
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Quel est l'ordre de grandeur de l'écart entre vent horizontal et le vent géostrophique ?</p>
<p><b>2) </b>D'après le champ de vitesse verticale de grande échelle, où peut-on s'attendre à observer de la divergence en altitude associée à de la convergence en basses couches ?</p>
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

# Champ de divergence

In [ ]:
fig = plt.figure(figsize=(10., 17.))
fig.suptitle('Geopotential height and divergence at '+str(ulev)+' hPa : '+date, fontsize=16)

ax = fig.add_subplot(311, projection=projection)
ax.set_title('Horizontal divergence at '+str(ulev)+ 'hPa and vertical motion at '+str(wlev)+ 'hPa',
             loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
cf = ax.contourf(lon, lat, div*10**6, levels_div, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('10$^{-6}$ s$^{-1}$')
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
c2 = ax.contour(lon, lat, w, levels_wpos[levels_wpos != 0], colors='brown', linewidths=1,
                transform=ccrs.PlateCarree()) 
c3 = ax.contour(lon, lat, w, levels_wneg[levels_wneg != 0], colors='green', linewidths=1,
                linestyle="--", transform=ccrs.PlateCarree()) 
ax.clabel(c2, fmt='%4.2f', fontsize=10)
ax.clabel(c3, fmt='%4.2f', fontsize=10)

ax = fig.add_subplot(312, projection=projection)
ax.set_title('Geostrophic divergence at '+str(ulev)+ 'hPa and vertical motion at '+str(wlev)+ 'hPa',
             loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
cf = ax.contourf(lon, lat, divg*10**6, levels_div, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('10$^{-6}$ s$^{-1}$')
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
c2 = ax.contour(lon, lat, w, levels_wpos[levels_wpos != 0], colors='brown', linewidths=1,
                transform=ccrs.PlateCarree()) 
c3 = ax.contour(lon, lat, w, levels_wneg[levels_wneg != 0], colors='green', linewidths=1,
                linestyle="--", transform=ccrs.PlateCarree()) 
ax.clabel(c2, fmt='%4.2f', fontsize=10)
ax.clabel(c3, fmt='%4.2f', fontsize=10)

ax = fig.add_subplot(313, projection=projection)
ax.set_title('Ageostrophic divergence at '+str(ulev)+ 'hPa and vertical motion at '+str(wlev)+ 'hPa',
             loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
cf = ax.contourf(lon, lat, diva*10**6, levels_div, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('10$^{-6}$ s$^{-1}$')
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
c2 = ax.contour(lon, lat, w, levels_wpos[levels_wpos != 0], colors='brown', linewidths=1,
                transform=ccrs.PlateCarree()) 
c3 = ax.contour(lon, lat, w, levels_wneg[levels_wneg != 0], colors='green', linewidths=1,
                linestyle="--", transform=ccrs.PlateCarree()) 
ax.clabel(c2, fmt='%4.2f', fontsize=10)
ax.clabel(c3, fmt='%4.2f', fontsize=10)

figname=dir_figs+'Z_div_'+str(zlev)+'_'+date
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b>Les ascendances de grande échelle sont-elles liées aux convergences-divergences du vent géostrophique ? Justifier.</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

# Champ de tourbillon

In [ ]:
fig = plt.figure(figsize=(10., 17.))
fig.suptitle('Geopotential height winds and vorticity at '+str(ulev)+' hPa', fontsize=16)

ax = fig.add_subplot(311, projection=projection)
ax.set_title('Geopotential height, relative vorticity and horizontal winds at '+str(ulev)+ 'hPa',
             loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
cf = ax.contourf(lon, lat, vort*10**5, levels_vor, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('10$^{-5}$ s$^{-1}$')
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                u[wind_slice, wind_slice], v[wind_slice, wind_slice],
                color='black', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)

ax = fig.add_subplot(312, projection=projection)
ax.set_title('Geopotential height, geostrophic vorticity and geostrophic wind at '+str(ulev)+' hPa',
             loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
cf = ax.contourf(lon, lat, vortg*10**5, levels_vor, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('10$^{-5}$ s$^{-1}$')
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                ug[wind_slice, wind_slice], vg[wind_slice, wind_slice],
                color='black', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)

ax = fig.add_subplot(313, projection=projection)
ax.set_title('Geopotential height, ageostrophic vorticity and ageostrophic wind at '+str(ulev)+ 'hPa',
             loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
cf = ax.contourf(lon, lat, vorta*10**5, levels_vor, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('10$^{-5}$ s$^{-1}$')
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                ua[wind_slice, wind_slice], va[wind_slice, wind_slice],
                color='black', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)

figname=dir_figs+'Z_vort_'+str(zlev)+'_'+date
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b>Le tourbillon géostrophique est-il une bonne approximation du tourbillon relatif ?</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

# Lien circulation-tourbillon

<div class="alert alert-warning">
<b>Choisir une zone de 2.5x2.5° dans une région de tourbillon relatif cyclonique. </b>
</div>

In [ ]:
lon1=int(input("Longitude W : "))
lon2=int(input("Longitude E : "))
lat1=int(input("Latitude S : "))
lat2=int(input("Latitude N : "))

In [ ]:
fig = plt.figure(figsize=(15., 8.))
fig.suptitle('Geopotential height winds and vorticity at '+str(ulev)+' hPa', fontsize=16)
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential height, relative vorticity and horizontal winds at '+str(ulev)+' hPa',
             loc='center', fontsize=10)
plot_background(ax)
box = sgeom.box(minx=lon1, maxx=lon2, miny=lat1, maxy=lat2)
ax.add_geometries([box], projection, facecolor='none', edgecolor='green', linewidths=3)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, vort*10**5, levels_vor, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('10$^{-5}$ s$^{-1}$')
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                u[wind_slice, wind_slice], v[wind_slice, wind_slice],
                color='black', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)

figname=dir_figs+'Z_vort_area_'+str(zlev)+'_'+date
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-danger">
<b>La fonction area calcule ici la surface (en m²) incluse dans la zone choisie. Calculer le tourbillon relatif moyen dans la zone choisie. Connaissant le lien entre circulation et tourbillon relatif moyen, en déduire la circulation autour du circuit. </b>
</div>

In [ ]:
obj = {'type':'Polygon','coordinates':[[[lon1,lat1],[lon1,lat2],[lon2,lat2],[lon2,lat1],[lon1,lat1]]]}
surface=area(obj)
print('Surface incluse dans la zone choisie (m²) : ', surface)